# S&P500 historical 20year data processing 
### In this file we are performing data preparation for training and test datasets for algo trading models.

In [44]:
# Imports
import pandas as pd
from pathlib import Path
import hvplot.pandas
from pandas.tseries.offsets import DateOffset


In [45]:
# Import the sp500 dataset into a Pandas Dataframe
trading_df = pd.read_csv(
    Path("../data/SP500_Data.csv"), 
    index_col="Date",
    infer_datetime_format=True, 
    parse_dates=True
    
)
# Review the DataFrame
trading_df.head()

,Price,Open,High,Low,Change %
Date,,,,,
2002-07-03,954.0,948.1,954.3,934.9,0.62%
2002-07-05,989.0,965.8,989.1,954.0,3.67%
2002-07-08,977.0,989.0,993.6,972.9,-1.21%
2002-07-09,952.8,977.0,979.6,951.7,-2.48%
2002-07-10,920.5,952.8,956.3,920.3,-3.39%


In [46]:
trading_df.shape

(5000, 5)

In [47]:
# Calculate the daily returns using the closing prices and the pct_change function
trading_df["actual_returns"] = trading_df["Price"].pct_change()

# Drop all NaN values from the DataFrame
trading_df = trading_df.dropna()

# Review the DataFrame
display(trading_df.head())
display(trading_df.tail())

,Price,Open,High,Low,Change %,actual_returns
Date,,,,,,
2002-07-05,989.0,965.8,989.1,954.0,3.67%,0.036688
2002-07-08,977.0,989.0,993.6,972.9,-1.21%,-0.012133
2002-07-09,952.8,977.0,979.6,951.7,-2.48%,-0.024770
2002-07-10,920.5,952.8,956.3,920.3,-3.39%,-0.033900
2002-07-11,927.4,920.5,929.2,900.9,0.75%,0.007496


,Price,Open,High,Low,Change %,actual_returns
Date,,,,,,
2022-05-05,4152.38,4255.50,4256.39,4105.51,-3.44%,-0.034368
2022-05-06,4123.34,4128.17,4157.69,4067.91,-0.70%,-0.006994
2022-05-09,3991.24,4081.27,4081.27,3975.48,-3.20%,-0.032037
2022-05-10,4001.05,4035.18,4068.82,3958.17,0.25%,0.002458
2022-05-11,3935.18,3990.08,4049.09,3928.82,-1.65%,-0.016463


In [48]:
# Define a window size of 12
short_window = 12

# Create a simple moving average (SMA) using the short_window and assign this to a new columns called sma_fast
trading_df["sma_fast"] = trading_df["Price"].rolling(window=short_window).mean()

In [49]:
# Define a window size of 50
long_window = 100

# Create a simple moving average (SMA) using the long_window and assign this to a new columns called sma_slow
trading_df["sma_slow"] = trading_df["Price"].rolling(window=long_window).mean()

In [50]:
trading_df.head()

,Price,Open,High,Low,Change %,actual_returns,sma_fast,sma_slow
Date,,,,,,,,
2002-07-05,989.0,965.8,989.1,954.0,3.67%,0.036688,NaN,NaN
2002-07-08,977.0,989.0,993.6,972.9,-1.21%,-0.012133,NaN,NaN
2002-07-09,952.8,977.0,979.6,951.7,-2.48%,-0.024770,NaN,NaN
2002-07-10,920.5,952.8,956.3,920.3,-3.39%,-0.033900,NaN,NaN
2002-07-11,927.4,920.5,929.2,900.9,0.75%,0.007496,NaN,NaN


In [51]:
# Drop the NaNs using dropna()
trading_df = trading_df.dropna()

In [52]:
# Assign a copy of the sma_fast and sma_slow columns to a new DataFrame called X
X = trading_df[["sma_fast", "sma_slow"]].copy()

# Display sample data
display(X.head())
display(X.tail())

,sma_fast,sma_slow
Date,,
2002-11-22,902.383333,885.746
2002-11-25,904.908333,885.185
2002-11-26,906.458333,884.548
2002-11-27,911.683333,884.409
2002-11-29,916.125000,884.567


,sma_fast,sma_slow
Date,,
2022-05-05,4248.584167,4472.2574
2022-05-06,4220.575000,4466.7993
2022-05-09,4187.040000,4460.3708
2022-05-10,4164.479167,4453.2829
2022-05-11,4134.400833,4445.9480


In [53]:
## Strategy with buy...hold...sell

# Create a new column in the trading_df called signal setting its value to zero.
#trading_df["signal"] = 0

# Create a value to hold the initial trade signal
#trade_signal = 0

#for index, row in trading_df.iterrows():
      
#    if (row["actual_returns"] >= 0) and (trade_signal < 1):
#        trading_df.loc[index, "signal"] = 1
#        trade_signal += 1
#    else:
#        if row["actual_returns"] < 0 and (trade_signal > 0):
#            trading_df.loc[index, "signal"] = -1
#            trade_signal = 0
        

#y = trading_df["signal"].copy()      

#trading_df

In [54]:
## Create the target set

# Create a new column in the trading_df called signal setting its value to zero.
trading_df["signal"] = 0

# Create the signal to buy
trading_df.loc[(trading_df["actual_returns"] >= 0), "signal"] = 1

# Create the signal to sell
trading_df.loc[(trading_df["actual_returns"] < 0), "signal"] = -1

# Copy the new signal column to a new Series called y.
y = trading_df["signal"].copy()

## Creating the Training Datasets

In [55]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

# Select the ending period for the training data with an offset of 4 years
training_end = X.index.min() + DateOffset(years=4)

# Display the training end date
print(training_end)

2002-11-22 00:00:00
2006-11-22 00:00:00


In [56]:
# Generate the X_train and y_train DataFrames
X_long_train = X.loc[training_begin:training_end]
y_long_train = y.loc[training_begin:training_end]

# Display sample data
display(X_long_train.head())
display(X_long_train.tail())

,sma_fast,sma_slow
Date,,
2002-11-22,902.383333,885.746
2002-11-25,904.908333,885.185
2002-11-26,906.458333,884.548
2002-11-27,911.683333,884.409
2002-11-29,916.125000,884.567


,sma_fast,sma_slow
Date,,
2006-11-16,1381.749167,1315.2835
2006-11-17,1384.531667,1316.8355
2006-11-20,1387.295000,1318.1118
2006-11-21,1390.504167,1319.4379
2006-11-22,1392.696667,1320.6969


In [57]:
display(y_long_train.head())
display(y_long_train.tail())

Date
2002-11-22   -1
2002-11-25    1
2002-11-26   -1
2002-11-27    1
2002-11-29   -1
Name: signal, dtype: int64

Date
2006-11-16    1
2006-11-17    1
2006-11-20   -1
2006-11-21    1
2006-11-22    1
Name: signal, dtype: int64

## Creating the long term testing Datasets

In [58]:
# Generate the X_long_test and y_long_test DataFrames

X_long_test = X.loc[training_end:]
y_long_test = y.loc[training_end:]

# Display sample data
# Display sample data
display(X_long_test.head())
display(X_long_test.tail())

,sma_fast,sma_slow
Date,,
2006-11-22,1392.696667,1320.6969
2006-11-24,1394.205833,1321.9973
2006-11-27,1393.892500,1323.0761
2006-11-28,1394.591667,1324.2885
2006-11-29,1396.140000,1325.6099


,sma_fast,sma_slow
Date,,
2022-05-05,4248.584167,4472.2574
2022-05-06,4220.575000,4466.7993
2022-05-09,4187.040000,4460.3708
2022-05-10,4164.479167,4453.2829
2022-05-11,4134.400833,4445.9480


In [59]:
display(y_long_test.head())
display(y_long_test.tail())

Date
2006-11-22    1
2006-11-24   -1
2006-11-27   -1
2006-11-28    1
2006-11-29    1
Name: signal, dtype: int64

Date
2022-05-05   -1
2022-05-06   -1
2022-05-09   -1
2022-05-10    1
2022-05-11   -1
Name: signal, dtype: int64

### Generate the 5 years data X_short_test and y_short_test DataFrames

In [60]:
short_testing_begin = X.index.max() - DateOffset(years=5)
short_testing_begin

Timestamp('2017-05-11 00:00:00')

In [61]:
X_short_test = X.loc[short_testing_begin:]
y_short_test = y.loc[short_testing_begin:]

# Display sample data
display(X_short_test.head())
display(X_short_test.tail())
display(y_short_test.head())
display(y_short_test.tail())

,sma_fast,sma_slow
Date,,
2017-05-11,2392.269167,2331.6940
2017-05-12,2392.556667,2333.0223
2017-05-15,2393.685833,2334.4202
2017-05-16,2395.058333,2335.7193
2017-05-17,2392.450000,2336.6378


,sma_fast,sma_slow
Date,,
2022-05-05,4248.584167,4472.2574
2022-05-06,4220.575000,4466.7993
2022-05-09,4187.040000,4460.3708
2022-05-10,4164.479167,4453.2829
2022-05-11,4134.400833,4445.9480


Date
2017-05-11   -1
2017-05-12   -1
2017-05-15    1
2017-05-16   -1
2017-05-17   -1
Name: signal, dtype: int64

Date
2022-05-05   -1
2022-05-06   -1
2022-05-09   -1
2022-05-10    1
2022-05-11   -1
Name: signal, dtype: int64

In [62]:
# Generate the X_short_train and y_short_train DataFrames for the previous 3 years of short testing dataset
short_testing_end = short_testing_begin
short_testing_begin =  short_testing_end - DateOffset(years=2)

In [63]:
X_short_train = X.loc[short_testing_begin:short_testing_end]
y_short_train = y.loc[short_testing_begin:short_testing_end]

# Display sample data
display(X_short_train.head())
display(y_short_train.head())
display(X_short_train.tail())
display(y_short_train.tail())

,sma_fast,sma_slow
Date,,
2015-05-11,2102.962500,2072.5771
2015-05-12,2101.415000,2073.8409
2015-05-13,2100.545000,2074.6968
2015-05-14,2101.073333,2075.2955
2015-05-15,2102.396667,2075.8163


Date
2015-05-11   -1
2015-05-12   -1
2015-05-13   -1
2015-05-14    1
2015-05-15    1
Name: signal, dtype: int64

,sma_fast,sma_slow
Date,,
2017-05-05,2382.012500,2326.2302
2017-05-08,2385.640833,2327.6544
2017-05-09,2389.660000,2328.9064
2017-05-10,2391.783333,2330.3699
2017-05-11,2392.269167,2331.6940


Date
2017-05-05    1
2017-05-08    1
2017-05-09   -1
2017-05-10    1
2017-05-11   -1
Name: signal, dtype: int64

## Create train and test data files

In [64]:
X_long_train.to_csv("../data/X_long_train.csv")
y_long_train.to_csv("../data/y_long_train.csv")

X_long_test.to_csv("../data/X_long_test.csv")
y_long_test.to_csv("../data/y_long_test.csv")

X_short_train.to_csv("../data/X_short_train.csv")
y_short_train.to_csv("../data/y_short_train.csv")

X_short_test.to_csv("../data/X_short_test.csv")
y_short_test.to_csv("../data/y_short_test.csv")